In [1]:
import dill
import pandas as pd
import jax.numpy as jnp
import os

burn_in = 500

In [2]:
# Needs to be path where the .csv with hMFC output are
csv_path = 'C:/Users/u0141056/OneDrive - KU Leuven/PhD/PROJECTS/Confidence and hMFC/Analysis/3. hMFC/3.2 Data with criterion fluctuations/individual datasets'
os.chdir(csv_path)

# Path where .dil files are
base_dilpath = 'C:/Users/u0141056/OneDrive - KU Leuven/PhD/PROJECTS/Confidence and hMFC/Analysis/3. hMFC/3.3 Dil files'

# Path where new .csv files have to be saved
new_csv_path = 'C:/Users/u0141056/OneDrive - KU Leuven/PhD/PROJECTS/Confidence and hMFC/Analysis/3. hMFC/3.4 Data with a and sigma_sq'

dirs = ['Adler_2018_Expt3',
 'Adler_2018_Expt1_taskA',
 'Adler_2018_Expt1_taskB',
 'Denison_2018',
 'Recht_unpub',
 'VanBoxtel_2019_Expt1',
 'VanBoxtel_2019_Expt2',
 'Law_unpub',
 'Yeon_unpub_Exp2',
 'Siedlecka_2021',
 'Shekhar_2021',
 'CalderTravis_unpub',
 'Filevich_unpub',
 'Maniscalco_2017_expt2',
 'Wang_2018']

In [3]:
for data_dir in dirs:
        
    print(data_dir)
            
    csvpath = csv_path + '/' + data_dir + '_with_criterion_fluctuations.csv'
        
    data = pd.read_csv(csvpath)   
        
    dilpath = base_dilpath + '/' + data_dir + '.dil'
    
    g = globals()
    with open(dilpath,'rb') as file:
        list_of_variable_names = dill.load(file)  # Get the names of stored objects
        for variable_name in list_of_variable_names:
            g[variable_name] = dill.load(file)    # Get the objects themselves
    
    data_summarized = data.groupby('subj').agg({'conf': ['mean', 'std'], 'criterion_fluctuations': 'std'}).reset_index()

    # Renaming columns for clarity
    data_summarized.columns = ['subj', 'mean_conf', 'sd_conf', 'sd_criterion_fluctuations']
    
    # Add posterior mean of a for each subject
    num_subjects = data_summarized.shape[0]
    
    mean_a = jnp.mean(posterior_samples_a[:,burn_in:,:].reshape(-1, num_subjects), axis=0)
    data_summarized.insert(4, 'a', mean_a)
    
    # Add posterior mean of sigma_sq for each subject
    mean_sigmasq = jnp.mean(posterior_samples_sigmasq[:,burn_in:,:].reshape(-1, num_subjects), axis=0)
    data_summarized.insert(5, 'sigmasq', mean_sigmasq)
   
    # Add posterior mean of mu_x for each subject
    mean_mu_x = jnp.mean(posterior_samples_mu_x[:,burn_in:,:].reshape(-1, num_subjects), axis=0)
    data_summarized.insert(6, 'mu_x', mean_mu_x)
    
    # Add name of study
    data_summarized.insert(7, 'study', data_dir)
    
    # Save data
    data_summarized.to_csv(new_csv_path + "/" + data_dir + '_with_a_sigmasq.csv')
    
    # delete other vars
    variables_to_keep = ['dirs', 'dilpath', 'new_csv_path', 'variables_to_keep', 'csv_path']

    # Get a dictionary of local variables
    local_vars = locals()
    
    # Create a new dictionary with only the variables to keep
    new_vars = {key: value for key, value in local_vars.items() if key in variables_to_keep or key.startswith('__')}
    
    # Update the local namespace with the new dictionary
    local_vars.update(new_vars)



Adler_2018_Expt3
Adler_2018_Expt1_taskA
Adler_2018_Expt1_taskB
Denison_2018
Recht_unpub
VanBoxtel_2019_Expt1
VanBoxtel_2019_Expt2
Law_unpub
Yeon_unpub_Exp2
Siedlecka_2021
Shekhar_2021
CalderTravis_unpub
Filevich_unpub
Maniscalco_2017_expt2
Wang_2018
